# Title: Use Machine Learning and Python to Develop Software, for Detect Financial Fraud in Text Messages 
Subject: ITW602 Advanced Information Technology - Work Integrated  

Group: 14
Student:  Carlos Carvajal
Id A00133955
Master of Software Engineering

Professor:
PhD Amr van den Adel

Torrens Univerity
Australia
Trimester 3 HE, 2025 (September 15 - December 7)
Date: dd-mm-yyyy


# 1 Business Understanding (Introductions)

1. Business Understanding (Introductions)
The problem context, objectives, scope, etc. are defined.
2. Data Understanding
The database is accessed (Chapman et al., 2000; kaggle, n.d.), the data type and content are analyzed, the data is checked for data issues, and the data is visualized.
3. Data Preparation
The data is prepared for processing, labels are converted into categorical data, stopwords are removed, and the data is divided into tokenization, stemming, and lemmatization. Finally, the data is prepared for training and testing.
4. Modeling
The N-Gram mathematical models are integrated with Python.
5. Evaluation
A qualitative and quantitative analysis of performance, as well as the precision and accuracy of the results, is performed using the confusion matrix.
6. Deployment
A test is performed using the Python shell (prototype), with a real-life example, and the results are analyzed.

# 2. Data Understanding

## Import Pythons 's Libraries Required

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import csv
import re
import random
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split

# phishing_pipeline.py  (run in your environment)
import os
import re
import joblib

# ML tools
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.utils import resample

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
print(' Test Run Ok 2.1')

 Test Run Ok 2.1


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Load the Dataset into a Pandas DataFrame
Access the page where the data is located, download the computer, and connect to Python to analyze its consistency, verify the data type, and repair errors.
Upload file.csv from local computer
Datase Reference: Salem, A. (2025). Spam_email_prediction. Kaggle - Dataset. https://www.kaggle.com/code/ayahsalemalshdefat/spam-email-prediction/notebook

In [3]:
# -------------------------
# Config / paths
# -------------------------
DATA_PATH = "./data/phishing_emails.csv"   # <- change to actual file from Kaggle
MODEL_OUT = "./best_phishing_model.joblib"
VECT_OUT = "./tfidf_vectorizer.joblib"

# -------------------------
# 1) Load dataset
# -------------------------
DATA_PATH = "phishing_email.csv"
#df = pd.read_csv('email.csv', encoding= "ISO-8859-1" )
df = pd.read_csv(DATA_PATH, encoding='utf-8', low_memory=False)

print("Columns:", df.columns.tolist())
# find columns with text/labels, adjust below:
# — assume label column might be named 'label' or 'Category' or 'class'
# Detect columns
label_col_candidates = ['label', 'Label', 'Category', 'category', 'class', 'target']
text_col_candidates  = ['text', 'message', 'Message', 'email', 'body', 'content', 'text_combined']

label_col = next((c for c in label_col_candidates if c in df.columns), None)
text_col  = next((c for c in text_col_candidates  if c in df.columns), None)

if not label_col or not text_col:
    raise RuntimeError(f"Could not find label/text column. Found: label={label_col}, text={text_col}")

# Keep and rename
df = df[[text_col, label_col]].rename(columns={
    text_col: 'text',
    label_col: 'label'
}).copy()
print(' Test 2.2 Run OK')


Columns: ['text_combined', 'label']
 Test 2.2 Run OK


## Check dataset file.csv

In [4]:
 # Upload file.csv from local computer
df = pd.read_csv(DATA_PATH, encoding='utf-8', low_memory=False)

print('df')
print(' ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ')
print('\n')

print('head')
df.head()
print( df.head())
print('---------------------------------------------------------')
print('\n')

print('columns names')
print(df.columns)
print('---------------------------------------------------------')
print('\n')

print('Count of columns :', len(df.columns))
print('Count of rows :  ', len(df))
print('---------------------------------------------------------')
print('\n')

print('infor')
print(df.info())
print('---------------------------------------------------------')
print('\n')

print(' tail ')
df.tail()
print('---------------------------------------------------------')
print('\n')

print('shape')
print(df.shape)
print('---------------------------------------------------------')
print('\n')

print('---------------------------------------------------------')
print('\n')
print('length of data is', len(df))

print('types')
print(df.dtypes)
print('---------------------------------------------------------')
print('\n')

print('describe')
print(df.describe())
print('---------------------------------------------------------')
print('\n')

print('isnull()')
print(df.isnull())
print('---------------------------------------------------------')
print('\n')

#Use Pandas to explore and clean up your tabular data
print('isnull().sum()')
print(df.isnull().sum())
print('---------------------------------------------------------')
print('\n')
# Python, check if there are missing cells

print('isnull().values.any()')
print(df.isnull().sum())
print('---------------------------------------------------------')
print('\n')

print('eq('').values.any()')
print(df.eq('').values.any())
print('---------------------------------------------------------')
print('\n')

print('---------------------------------------------------------')
print('Check which columns have NaN values')
# Check which columns have NaN values
columns_with_nan = df.isnull().any()

print('---------------------------------------------------------')
print('')
# Get actual column names from the DataFrame
columns_to_count = df.columns

# Now you can use value_counts on all columns
df[columns_to_count].value_counts()

print(' Test Run Ok 2.3')

df
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 


head
                                       text_combined  label
0  hpl nom may 25 2001 see attached file hplno 52...      0
1  nom actual vols 24 th forwarded sabrae zajac h...      0
2  enron actuals march 30 april 1 201 estimated a...      0
3  hpl nom may 30 2001 see attached file hplno 53...      0
4  hpl nom june 1 2001 see attached file hplno 60...      0
---------------------------------------------------------


columns names
Index(['text_combined', 'label'], dtype='object')
---------------------------------------------------------


Count of columns : 2
Count of rows :   82486
---------------------------------------------------------


infor
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82486 entries, 0 to 82485
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text_combined  82486 non-null  object
 1   label       

# 3. Data Preparation
convert everything to lowercase strings, Since the processor and compiler can be context-sensitive, all columns are converted to lower case
Check Category's values.
Category [legit, phishing] in lowercase format
To the words 'phishing', 'phish', 'spam', 'scam' they are given format 'phishing
## Numeric target
'y' = map({'legit': 0, 'phishing': 1})

In [5]:
# -------------------------
# 2) Clean labels robustly
# -------------------------

# convert everything to lowercase strings
df['label'] = df['label'].astype(str).str.strip().str.lower()

# numeric datasets (like yours)
df['label'] = df['label'].replace({
    '0': 'legit',
    '1': 'phishing'
})

# map typical variants
label_map = {
    'phishing': 'phishing',
    'phish': 'phishing',
    'spam': 'phishing',
    'scam': 'phishing',
    'ham': 'legit',
    'legit': 'legit',
    'legitimate': 'legit',
    'benign': 'legit'
}

df['label'] = df['label'].apply(lambda s: label_map.get(s, 'unknown'))

# keep only valid rows
df = df[df['label'].isin(['phishing','legit'])].copy()

# numeric target
df['y'] = df['label'].map({'legit': 0, 'phishing': 1})

print("legit: 0\n phishing: 1")
print("Label counts:\n", df['y'].value_counts())
print(' Test 3.1 Run OK -')


legit: 0
 phishing: 1
Label counts:
 y
1    42891
0    39595
Name: count, dtype: int64
 Test 3.1 Run OK -


 ## Text preprocessing
 Text messages are being refined.
 Some words are removed from the stopwords; since their presence can affect the detection of phishing messages.
 Unnecessary characters that do not affect the semantics of the text are removed.
 The URLs are removed
 Messages are tokenized basic units and lemmatized

In [6]:

wn = WordNetLemmatizer()
builtin_stop = set(stopwords.words('english'))

# remove these from stopword removal because they're useful for phishing detection
keep_tokens = {'account','password','verify','suspend','suspended','login','bank','click','link','update','confirm','security'}
# final stoplist = builtin_stop minus keep_tokens
stoplist = builtin_stop - keep_tokens

def preprocess_text(text):
    if not isinstance(text, str): 
        return ''
    text = text.lower()
    # optional: normalize unicode
    text = re.sub(r'https?://\S+',' ', text)   # remove URLs (or keep them if you want to extract domain features)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)   # keep alphanumerics (numbers can help)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stoplist]
    tokens = [wn.lemmatize(t) for t in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['text_combined'].apply(preprocess_text)
#print(df)
print(' Test 3.2 Run OK -')

 Test 3.2 Run OK -


##  Balance classes 
In machine learning models, in many cases, if the models are trained with unbalanced data, there is a tendency to predict the one with the larger number.

legit: 0
phishing: 1
Label counts:
0 39595
1 42891

Since the phishing class (1) is smaller than the legit class (0), a balance is created.
The following code snippet checks if there is an imbalance. Then, since legitimate labels are in the minority, they are balanced with phishing labels.

In [7]:
# -------------------------
# 4) Balance classes (upsample phishing if minority)
# -------------------------
print("Before balancing:", df['y'].value_counts())
n_phish = df['y'].sum()
n_legit = len(df) - n_phish

if n_phish == 0:
    raise RuntimeError("No phishing samples found — add phishing data!")

if n_phish < n_legit:
    phishing = df[df['y']==1]
    legit = df[df['y']==0]
    phishing_upsampled = resample(phishing, replace=True, n_samples=len(legit), random_state=42)
    df_bal = pd.concat([legit, phishing_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)
else:
    df_bal = df.copy()

print("legit: 0\n phishing: 1")
print("After balancing:", df_bal['y'].value_counts())
print(' Test 3.3 Run OK -')
# -------------------------


Before balancing: y
1    42891
0    39595
Name: count, dtype: int64
legit: 0
 phishing: 1
After balancing: y
1    42891
0    39595
Name: count, dtype: int64
 Test 3.3 Run OK -


## Train/test split and TF-IDF
## Selecting the input (X) and target (y)
 Splitting into train/test sets
 75% training and 25% testing.
 stratify=y ensures both train and test have the same class proportions.
 random_state=42 ensures reproducibility.
 They are vectorized using NGram
TF-IDF (Term Frequency – Inverse Document Frequency)
Assigns numerical characteristics to text messages so machine learning can better process the data.

TF: What is the frequency of a word?
IDF: How rare is the word?

N-grams: These are important because phrases, not just words, are used in message semantics.

Example:
ngram_range=(1,3)
Unigram: "update"
Bigram: "update account"
Trigram: "verify your account"

lowercase=True: Converts all text to lowercase.
max_features=50000: Preserves the 50,000 most important features.

tfidf.transform(X_test): Transforms the test data using the same vocabulary.

In [8]:
# 5) Train/test split and TF-IDF
# -------------------------
X = df_bal['clean_text']
y = df_bal['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

# TF-IDF: use 1-3 grams (unigrams, bigrams, trigrams)
tfidf = TfidfVectorizer(ngram_range=(1,3), lowercase=True, max_features=50000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)
print(' Test 3.4 Run OK -')

 Test 3.4 Run OK -


# 4. Modeling

## The ML's mathematical algorithms are implemented.
## The mathematical models are evaluated.
## Logistic Regression
## max_iter=2000 → give the model enough iterations to converge
## class_weight='balanced' → correct class imbalance
## solver='liblinear' → reliable solver for small/medium datasets and binary classification

## Naive Bayes
## Simple and fast text classifier
## Works well with TF-IDF or CountVectorizer

## Train Each Model and Evaluate It

In [ ]:
# -------------------------
# 6) Train models and evaluate
# -------------------------
'''
models = {
    'LogisticRegression': LogisticRegression(max_iter=2000, class_weight='balanced', solver='liblinear'),
    'NaiveBayes': MultinomialNB(),
    'SVC': SVC(probability=True, class_weight='balanced', kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
} 
'''
models = {
    'LogisticRegression': LogisticRegression(max_iter=2000, class_weight='balanced', solver='liblinear'),
    'NaiveBayes': MultinomialNB(),
}

results = {}
for name, m in models.items():
    m.fit(X_train_tfidf, y_train)
    preds = m.predict(X_test_tfidf)
    print(f"\n=== {name} ==============================")
    print("Accuracy:", accuracy_score(y_test, preds))
    #print(classification_report(y_test, preds, digits=4))
    # cm = confusion_matrix(y_test, preds)
    # print("Confusion matrix:\n", cm)
    # results[name] = m
    confusion_matrixs=confusion_matrix(y_test,preds)
    sns.heatmap(confusion_matrixs,annot=True,fmt="1.0f",cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f"Confusion Matrix For {name}")
    plt.show()
    #print('\n')
      
    # Extract values
    TN, FP, FN, TP = confusion_matrixs.ravel()
    
    #print("")
    print(f"True Negative (TN) = {TN}")
    print(f"False Positive (FP) = {FP}")
    print(f"False Negative (FN) = {FN}")
    print(f"True Positive (TP) = {TP}")
    
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    print("Accuracy :" , accuracy)
    print('\n')
    #print(' \n Run Ok 1') 
    classification_reports=classification_report(y_test,preds)
    print(f'{name} classification_report:\n{classification_reports}')
    #print('\n')
    
print(' Test 4 Run OK')

# 5. Evaluation

In [ ]:

# choose best by F1 or accuracy (example use LogisticRegression)
best_name = max(models.keys(), key=lambda n: accuracy_score(y_test, models[n].predict(X_test_tfidf)))
best_model = models[best_name]
print("Best:", best_name)

# Save vectorizer and model
joblib.dump(tfidf, VECT_OUT)
joblib.dump(best_model, MODEL_OUT)
print("Saved:", VECT_OUT, MODEL_OUT)
print(' Test 5 Run OK')

# 6. Deployment

In [23]:
# -------------------------
# 7) Prediction helper
# -------------------------
def predict_email(text, model=best_model, vectorizer=tfidf):
    cleaned = preprocess_text(text)
    vector = vectorizer.transform([cleaned])
    pred = model.predict(vector)[0]
    prob = model.predict_proba(vector)[0] if hasattr(model, "predict_proba") else None
    return pred, prob
'''
# quick test
msg = "Your account will be suspended, verify your password now."
pred, prob = predict_email(msg)
label_map = {0: "legit", 1: "phishing"}
print("Prediction:", label_map[pred])
print("Probabilities:", prob)
'''
import tkinter as tk
from tkinter import messagebox

def run_gui():
    def diagnose():
        text = text_entry.get("1.0", tk.END).strip()
        pred, prob = predict_email(text)
        label_map = {0: "legit", 1: "phishing"}
        label = label_map[pred]
        probability = prob[pred]

        messagebox.showinfo("Diagnosis", f"{label.upper()} (prob: {probability:.4f})")

    window = tk.Tk()
    window.title("Phishing Message Detector")

    tk.Label(window, text="Enter message:").pack()

    text_entry = tk.Text(window, height=6, width=60)
    text_entry.pack()

    tk.Button(window, text="Check", command=diagnose).pack()

    window.mainloop()

run_gui()


print(' Test 6 Run OK')

 Test 3 Run OK
